In [147]:
# add root folder of the project to path
import sys
sys.path.insert(0, '..')

In [148]:
"""
export_folder = '../output/Notes'
asset_folder = '/Users/kirov/Library/Mobile Documents/iCloud~com~apple~iBooks/Documents/'
"""

"\nexport_folder = '../output/Notes'\nasset_folder = '/Users/kirov/Library/Mobile Documents/iCloud~com~apple~iBooks/Documents/'\n"

In [149]:
#"""
bib_path = '/Users/kirov/Library/Mobile Documents/iCloud~com~apple~iBooks/Documents/Knowledge.bib'
export_folder = '/Users/kirov/Library/Mobile Documents/iCloud~md~obsidian/Documents/Appraise/6-Humboldt/5-Materials/Literatures/'
asset_folder = '/Users/kirov/Library/Mobile Documents/iCloud~com~apple~iBooks/Documents/'
#"""

#### `.bib` entries loading

In [160]:
import bibtexparser

def load_bib(bib_path: str):
    with open(bib_path) as bibtex_file:
        bib_database = bibtexparser.load(bibtex_file)
    return bib_database

bib_database = load_bib(bib_path)

Entry type software not standard. Not considered.
Entry type software not standard. Not considered.
Entry type software not standard. Not considered.
Entry type software not standard. Not considered.
Entry type software not standard. Not considered.
Entry type software not standard. Not considered.
Entry type software not standard. Not considered.


#### Group tree parsing

In [152]:
import re

def parse_group_tree(tree_str: str) -> dict:
    # extract group level and name and combined as an order list
    lines = re.split('\n', tree_str)
    group_list = []

    for line in lines:
        if 'Group' in line:
            level = int(line[0])
            if level == 0:
                group_list.append({
                    'level': level,
                    'name': 'Root',
                    'node': {'entries': []},
                    })
                
            else:
                name = re.search(r"(?<=Group:)[^\\;]+", line).group()  # at current stage only english group names are supported
                group_list.append({
                    'level': level, 
                    'name': name,
                    'node': {'entries': []},
                    })

    # prepare a dictionary of all groups for fast accessing via group name
    group_dict = {}

    for group in group_list:
        group_dict[group['name']] = group['node']

    # parse the group tree
    for idx_current in range(len(group_list)):
        # search backward for the 1st higher level node as current node's source node
        for idx_previous in range(idx_current, -1, -1):
            if group_list[idx_previous]['level'] < group_list[idx_current]['level']:
                source_node = group_list[idx_previous]['node']
                current_node = group_list[idx_current]['node']
                current_name = group_list[idx_current]['name']
                source_node[current_name] = current_node
                break
    
    tree_root = group_list[0]['node']

    return group_dict, tree_root

tree_str = bib_database.comments[1]
group_dict, tree_root = parse_group_tree(tree_str)

#### Attach each group with the corresponding entries

In [153]:
def attach_entries_to_group(entries: list, group_dict: dict):
    for entry in entries:
        # if the entry has groups attr, attach it to the corresponding groups
        if 'groups' in entry.keys():
            groups = re.split(', ', entry['groups'])

            for group in groups:
                group_dict[group]['entries'].append(entry)

        # if the entry doesn't have groups attr, attach it to the root group
        else:
            group_dict['Root']['entries'].append(entry)

attach_entries_to_group(bib_database.entries, group_dict)

#### Generate Citation Text

In [154]:
# convert js citation scripts to python functions
from quickjs import Function

with open('citation_full.js') as file:
    js = ''.join(file.readlines())
    citation_full = Function('citation_full', js)

with open('citation.js') as file:
    js = ''.join(file.readlines())
    citation = Function('citation', js)

citation("article", "Wittek_A_2015", "Adam Wittek, Nicole M. Grosland, Grand Roman Joldes, Vincent Magnotta, Karol Miller", "From finite element meshes to clouds of points a review of methods for generation of computational biomechanics models for patient-specific applications", "", "", "Ann. Biomed. Eng.", "Springer Science and Business Media LLC", "", "", "2015", "44", "1", "3–15")

'Adam Wittek, et al. "[[From finite element meshes to clouds of points a review of methods for generation of computational biomechanics models for patient-specific applications]]". Ann. Biomed. Eng.. 2015, 44(1): 3–15.'

In [155]:
def param_assemble(entry: dict):
    param2key = {
        'type_str': 'ENTRYTYPE', 
        'cite_str': 'ID',
        'author_str': 'author', 
        'title_str': 'title', 
        'editor_str': 'editor', 
        'edition_str': 'edition',
        'container_str': 'container', 
        'publisher_str': 'publisher', 
        'school_str': 'school', 
        'addres_str': 'address',
        'year_str': 'year', 
        'vol_str': 'volume', 
        'num_str': 'number', 
        'page_str': 'pages'
    }
    
    param_list = []

    for param, key in param2key.items():
        if key in entry.keys():
            param_list.append(entry[key])
        else:
            param_list.append('')

    return param_list

In [156]:
def gen_citation(entry: dict):
    param_list = param_assemble(entry)
    return citation(*param_list)

gen_citation(bib_database.entries[3])

'Sun, et al. "[[Optimization method for the determination of mooney-rivlin material coefficients of the human breasts in-vivo using static and dynamic finite element models]]". 2019, 90: 615--625.'

In [157]:
def gen_citation_full(entry: dict):
    param_list = param_assemble(entry)
    return citation_full(*param_list)

gen_citation_full(bib_database.entries[3])

'Sun,  Yue and Chen,  Lihua and Yick,  Kit-lun and Yu,  Winnie and Lau,  Newman and Jiao,  Wanzhong. "[[Optimization method for the determination of mooney-rivlin material coefficients of the human breasts in-vivo using static and dynamic finite element models]]". 2019, 90: 615--625.'

#### Generate group pages

In [158]:
import os
import shutil

def export_group_pages(node: dict, name: str, source_name: str = "None", export_folder: str = 'output/Groups/'):
    # create group folder
    if os.path.exists(export_folder):
        shutil.rmtree(export_folder)
    os.mkdir(export_folder)

    # create group's markdown page
    md_path = os.path.join(export_folder, '{}.md'.format(name))
            
    with open(md_path, 'w') as file:
        # title section
        file.write("# {}\n".format(name))

        # roadmap section
        file.write("---\n")
        file.write("#### Roadmap\n\n")

        if name is not None:
            file.write("Source group\n")
            file.write("[[{}]]\n".format(source_name))

        file.write("Subgroups \n\n")

        for subgroup in node.keys():
            if subgroup != "entries":
                file.write("- [[{}]]\n".format(subgroup))

        # entries section
        file.write("---\n")
        file.write("#### Entries\n\n")
        file.write("| Type | Entry |\n")
        file.write("| --- | --- |\n")
        
        for entry in node['entries']:
            try:
                file.write("| {} | {} |\n".format(
                entry['ENTRYTYPE'],
                gen_citation(entry),
                ))
            except:
                print("entry information miss:")
                print(entry)

    # recurse to the subgroups
    for subgroup in node.keys():
        if subgroup != "entries":
            sub_folder = os.path.join(export_folder, subgroup)
            export_group_pages(node[subgroup], subgroup, name, sub_folder)

export_group_pages(node=tree_root, name='Root', export_folder=os.path.join(export_folder, 'Groups'))

#### Generate Literature Review Pages

In [159]:
def export_note_pages(entries: list, export_folder: str = 'output/Notes', asset_folder: str = '/Users/kirov/Library/Mobile Documents/iCloud~com~apple~iBooks/Documents/'):
    # create notes folder
    if os.path.exists(export_folder):
        shutil.rmtree(export_folder)
    os.mkdir(export_folder)

    # create markdown files for notes
    for entry in entries:
        try:
            md_path = os.path.join(export_folder, "{}.md".format(entry['title']))
            with open(md_path, 'w') as file:
                # title section
                file.write("### {}\n".format(entry['title']))
                if 'groups' in entry.keys():
                    group_str = entry['groups'].replace(", ", "]]. [[")
                    file.write("$\infty$ [[{}]]\n".format(group_str))

                if 'author' in entry.keys():
                    file.write("{}\n".format(entry['author']))
                if 'editor' in entry.keys():
                     file.write("Editors: {}\n".format(entry['editor']))

                # asset section
                file.write("---\n")
                file.write("#### Asset\n\n")
                file.write("> {}\n".format("{}\n".format(gen_citation_full(entry))))
                
                if 'file' in entry.keys():
                    file_path = "file:/{}".format(os.path.join(asset_folder, entry['file'][1:-4])).replace(" ", "%20")
                    file.write("- Open [local file]({})^[only available on desktop] or search it in [Apple Books](ibooks://search).\n".format(file_path))

                file.write("- Citation key\n")
                file.write("  ```\n")
                file.write("  {}\n".format(entry['ID']))
                file.write("  ```\n")

                file.write("Citation text^[The citation text presented here is only for adding brief literature citations referenced to this note in another Obsidian note, not for formal academic papers.]\n")
                file.write("  ```\n")
                file.write("  {}\n".format(gen_citation(entry)))
                file.write("  ```\n")

                # literature review section
                if 'comment' in entry.keys():
                    file.write("---\n")
                    file.write("#### Literature Review\n\n")
                    file.write("{}\n\n".format(entry['comment']))

        except:
                print("entry information miss:")
                print(entry)

export_note_pages(bib_database.entries, export_folder=os.path.join(export_folder, 'Notes'), asset_folder=asset_folder)

entry information miss:
{'url': 'https://www.ebook.de/de/product/30961446/marco_oechsner_a_first_introduction_to_the_finite_element_analysis_program_msc_marc_mentat.html', 'priority': 'prio2', 'pagetotal': '158', 'groups': 'Physical System', 'file': ':A First Introduction to the Finite Element Analysis Program Msc Marc_mentat.pdf:PDF', 'ean': '9783319719153', 'month': 'November', 'isbn': '9783319719153', 'year': '2017', 'title': 'A first introduction to the finite element analysis program msc marc/mentat', 'publisher': 'Springer-Verlag GmbH', 'author': 'Öchsner, Marco', 'ENTRYTYPE': 'book', 'ID': 'Oechsner_M_2017'}
